<a href="https://colab.research.google.com/github/zxc954/10924228.AI/blob/main/%E3%80%8C2023_04_21_OpenAI_Whisper_%E8%AA%9E%E9%9F%B3%E8%BE%A8%E8%AD%98_%E8%A3%BD%E4%BD%9C%E5%AD%97%E5%B9%95%E6%AA%94_v2_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OpenAI 的 Whisper 語音辨識測試 v.2

update: 2023.04.21 by gsyan
* 可使用 .txt 檔案放要辨識的清單。
* 可自訂字幕檔要儲存的資料夾。
* 可自訂是否要跳過已辨識過的。
* 改用 yt-dlp 下載影音檔案。

如果原本可正常使用，卻發生問題而中斷程序，可能是 Whisper 專案有所異動，可以到我的 Blogger 中留言反應，我再來除錯修正，Blogger 網址如下：
* https://gsyan888.blogspot.com/2023/02/openai-whisper-ipynb.html

OpenaAI 的 Whisper 是一個自動語音辨識系統，而且有開源，可以在底下的網址中找到:
* https://github.com/openai/whisper

結合 Whisper 和 pytube 或其它類似 yt-dlp 的工具，就可以將 Youtube 上的影片或播放清單擷取聲音、儲存語音檔後，進行語音辨識，並生成字幕檔。

目前在後面程式設定區塊中，語音來源路徑的「url」欄位中，可以填入 Youtube 的影片或影片清單網址；如果想馬上錄一段做測試，也可以利用線上錄音的網站 [Vocaroo](https://voca.roo/) (https://voca.roo/) 來錄音，再將它給的網址貼在「url」欄位中；如果是在電腦本機中的影片或聲音檔，則可以上載在左側資料夾中後，在「url」欄位中填入完整的檔名名稱。

接著將其它選項都設定好後，就可以在[程式區塊]中按「執行」的按鈕，開始進行語音辨識了。

Whisper 也可以辨識臺語，不過記得設定時，語言代碼「lang」的欄位要選「Chinese」比較保險，如果用「自動判斷」，有時會被當成非中文，而無法辨識出文字。

程式第一次執行時，因為要安裝及下載自動語音辨識所需要的資料，可能要稍等一下下。



In [2]:
#@title OpenAI Whisper 語音辨識並輸出字幕檔案程式 { vertical-output: true }

#@markdown <b>設定底下的自訂參數後，就可以按左側的執行鈕</b>

#@markdown <b>聲音檔的來源</b>，可以是網址(Youtube影片、撥放清單、 [Vocaroo](https://voca.roo/) 網址)，或是上載後的影片、聲音檔檔案名稱。<br />有多個可以放入 .txt 檔案中(例: list.txt)，一行一個網址或是檔案路徑。
url = "https://www.youtube.com/watch?v=T0rm3hwY8lk" #@param {type:"string"}
#@markdown <b>語音的語言代碼</b>
lang = 'Chinese' #@param ["Chinese", "English", "Japanese", "Korean", "自動判斷"]
#@markdown <b>輸出為哪一種格式</b>（.srt:字幕檔、.txt:純文字檔）
outputFormat = 'srt' #@param ['srt', 'txt']
#@markdown <b>輸出到哪個資料夾</b>(空白表示使用預設值)
output_path = '' #@param {type:"string"}
#@markdown <b>是否覆蓋已存在的辨識結果</b>(沒勾選會跳過已辨識過的)
overwrite_enable = True #@param { type: 'boolean' }
#@markdown <b>使用哪一種辨識模型</b>（small:快/普通，medium:慢/精準）
modelType = 'small' #@param ["small" , "medium","large"]
#@markdown <b>是否全部辨識完成，立即下載字幕檔</b>
start_downloading_immediately = True #@param { type: 'boolean' }
#@markdown <b>是否即時顯示語音辨識結果</b>
verbose = False #@param { type: 'boolean' }
#markdown <b>是否使用 yt-dlp 下載 Youtube 聲音檔</b><br />(如果較長的聲音檔下載發生問題時再勾選使用)
#YoutubeDL_enable = False #@param { type: 'boolean' }

#@markdown ---
#@markdown <center><p><font color="blue">以下為指令輸出內容</p></center>




# Install + Import + Config
try: import whisper
except:
  print('install whisper ...')
  ! pip -q install git+https://github.com/openai/whisper.git

#try: import yt_dlp
#except:
#  print('install ;yt-dlp ...')
#  ! pip -q install yt-dlp

#try: from pytube import YouTube
#except:
#  print('install pytube ...')
#  ! pip -q install pytube

try: from yt_dlp import YoutubeDL 
except :
  print('install yt_dlp ...')
  ! pip -q install yt_dlp

  
try: from slugify import slugify
except:
  print('install python-slugify ...')
  ! pip install python-slugify


import torch
import whisper
from whisper.utils import get_writer
#from pytube import YouTube
#from pytube import Playlist
from yt_dlp import YoutubeDL
import re
import os
import urllib.request
from slugify import slugify

import google

#import yt_dlp

#url = "https://voca.ro/15HVH0YvIaa6"
#url = 'https://www.youtube.com/watch?v=I4DZn4z8aRQ&list=PLelNvYGEtsV8TpwxL4t7GTTG-7qALZqol'
#url = 'https://www.youtube.com/watch?v=I4DZn4z8aRQ'
#url = 'vocaroo-台語.mp3'

#lang = 'Chinese' # '自動判斷'
#start_downloading_immediately = True

#modelType = 'small' # 'small' 'medium'
#outputFormat = 'txt' # 'srt' 'txt'

audioFile = 'source.mp3' #暫存的語音檔檔名
tempFolder = '.' #暫存的資料夾(工作目錄、下載的影音、剛轉好的文字檔)
#output_path = '.'
title = ''
textFileList = [] #記錄轉好的文字檔檔名清單

#檢查輸出文字檔的資料夾是否存在, 不存在就改用預設值
if re.sub('\s', '', output_path) == '' :
  #如果沒有輸入，就輸出到預設的暫存資料夾
  output_path = tempFolder
else :
  if os.path.exists(output_path) :
    if not os.path.isdir(output_path) :
      print(output_path + ' 不是資料夾哦! 改用預設值: '+tempFolder)
      output_path = tempFolder
  else :
    print('指定的輸出目錄 '+output_path+' 不存在, 改用預設值: '+tempFolder)
    output_path = tempFolder


# GPU or CPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def getYoutubePlaylistInfo_ydl(url) :
  """
  使用 yt_dlp 由 Youtbue 播放清單網址擷取各影片的資訊
  :param url: Youtube 影片播放清單網址
  """  
  ydl_opts = {
      'quiet':True,
      #'extract_flat':True
  }
  with YoutubeDL(ydl_opts) as ydl: 
    info_dict = ydl.extract_info(url, download=False)
    #return (info_dict.get('entries', None)) #entries 為清單中的影片, 'webpage_url' 為影片網址
    return info_dict

def getYoutubeTitle_ydl(url) :
  """
  使用 yt_dlp 由 Youtbue 網址擷取影片的標題字
  :param url: Youtube 影片網址
  """  
  ydl_opts = {
      'quiet':True
  }
  with YoutubeDL(ydl_opts) as ydl: 
    info_dict = ydl.extract_info(url, download=False)
    return(info_dict.get('title', None))

def getAudioFromYoutube_ydl(url, output_path, outputFilename):
  """
  使用 yt_dlp 由 Youtbue 網址下載影片的聲音並存檔
  :param url: Youtube 影片網址
  :param output_path: 儲存的資料夾名稱
  :param outputFilename: 聲音檔的檔名
  """
  #filename為去掉副檔名的路徑，以免變成 xxx.mp3.mp3
  filename = os.path.join(output_path, os.path.splitext(outputFilename)[0])
  ydl_opts = {
      'quiet':True,
      'outtmpl':filename,
      'keepvideo':False,
      'format': 'bestaudio/best',
      'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
      #  'preferredquality': '192',
      }],      
      'overwrites':True
  }
  with YoutubeDL(ydl_opts) as ydl: 
    ydl.download(url)

#def getAudioFromYoutube(url, output_path, filename) :
#  """
#  使用 pytube, 由 Youtbue 網址下載影片的聲音並存檔
#  :param url: Youtube 影片網址
#  :param output_path: 儲存的資料夾名稱
#  :param filename: 聲音檔的檔名
#  """
#  video = YouTube(url)
#  if not YoutubeDL_enable :    
#    video.streams.get_audio_only().download(output_path, filename)
#  else :
#    getAudioFromYoutube_ydl(url, output_path, filename)
#  return video.title
#

def getVocarooMP3URL(url) :
  """
  Get the MP3 URL from Vocaroo record share URL
  :param url: Vocaroo url
  """
  vocarooMP3Base = 'https://media.vocaroo.com/mp3/'
  regex = re.compile(r'(https\:\/\/voca\.ro|https\:\/\/vocaroo\.com)\/(\w{12})')
  match = regex.search(url)
  if match :
    url = vocarooMP3Base+match.group(2)
  return url

def transcribe(audioFilename, output_path, outputFilename, outputFormat) :
  """
  load whisper model and get transcribe result
  :param audioFilename: audio file to transcribe
  :param output_path: the folder to save result
  :param outputFilename: the filename(without the extension) to save result  
  :param outputFormat: the transcribe result format (thie extension of filename)
  """
  #print(audioFilename+' --> '+output_path+outputFilename+'.'+outputFormat)
  #return 1

  model = whisper.load_model(modelType, device=DEVICE)
  if lang=="自動判斷" :
    print('auto detect language')
    result = model.transcribe(audioFilename, fp16=False, verbose=verbose)    
  else :
    result = model.transcribe(audioFilename, fp16=False, verbose=verbose, language=lang)
  # save result to outputFormat file
  saveToFile(result, output_path, outputFilename, outputFormat)

def saveToFile(result, output_path, filename, fileType='srt') :
  """
  save whisper transcribe result to a file
  :param result: whisper transcribe result
  :param output_path: the folder to save result
  :param filename: the filename(without the extension) to save result  
  :param fileType: the transcribe result format (thie extension of filename)
  """
  # save SRT(full filename : output_path/filename.fileType)
  file_writer = get_writer(fileType, output_path) #whisper get_writer
  #file_writer(result, filename)
  #2023.04.12 因應可標定字的時間截記 word_timestamps 必須傳入 writer_args 參數
  writer_args = {'highlight_words': False, 'max_line_count': None, 'max_line_width': None}
  file_writer(result, filename, writer_args)


#
def getFilenameFromTitle(title) :
  """
  convert title to valid filename
  :param title: text to slugify
  """
  return slugify(title, allow_unicode=True, lowercase=False)

def getOutputTextFilename(filename) :
  fPath = os.path.relpath(output_path)
  return os.path.join(fPath, filename+'.'+outputFormat)

def isOutputTextFileExists(filename) :
  return os.path.exists(getOutputTextFilename(filename))

def removeAudioFile(filename) :
  if os.path.exists(filename) :
    os.remove(filename)

def convertNewLineFormat(filename) :
  if os.path.exists(filename) :
    f = open(filename, 'r+')
    data = f.read()
    data = re.sub('\r', '', data, re.MULTILINE) #remove all \r
    data = re.sub('\n', '\r\n', data, re.MULTILINE) #replace \n to \r\n
    f.seek(0)
    f.write(data)
    f.truncate()
    f.close()

def getAudioAndTranscribe(urlOrFile) :
  filename = audioFile #語音檔檔名(辨識用的暫存檔，用固定的名稱較方便)
  # 分網址或是Colab儲存空間檔案處理
  if re.search('https\:\/\/', urlOrFile) :
    isPlayList = False
    # Youtube 分播放清單跟單一影片處理
    if re.search('youtube\.|youtu\.', urlOrFile) :
      # 建立 Playlist 物件
      #pList = Playlist(urlOrFile)
      videoInfo = getYoutubePlaylistInfo_ydl(urlOrFile)
      isPlayList = True
      #try : title = pList.title
      try : videos = videoInfo['entries']
      except :
        isPlayList = False
      #處理 Youtube 播放清單
      if isPlayList :
        title = videoInfo['title']
        print('\n>>處理 Youtube 播放清單影片: '+title)
        #for video in pList.videos :
        for video in videos :
          #title = video.title #取得Youtube影片的標題字
          title = video['title'] #取得Youtube影片的標題字
          #filename = title+'.mp3'
          print('\n>>> '+title)
          #用影片的標題當主檔名，但需將不適合的字置換或去除
          #convert title to valid filename
          outputFilename = getFilenameFromTitle(title)
          #continue

          #如果設定辨識過的重新辨識(overwrite_enable), 或是輸出字幕檔不存在就呼叫whisper
          if not isOutputTextFileExists(outputFilename) or overwrite_enable :
            # delete the old audio file
            removeAudioFile(filename)
            # download audio from video stream
            #video.streams.get_audio_only().download(tempFolder, filename)
            print('>> 下載影音檔案中...')
            try: getAudioFromYoutube_ydl(video['webpage_url'], tempFolder, filename)
            except:
              print('>>>無法由網址下載影音')

            #print('\n>> 準備進行語音辨識...\n')
            if os.path.exists(filename) :
              print('>>>Whisper 辨識中...\n')
              # load whisper model and get transcribe result
              transcribe(filename, output_path, outputFilename, outputFormat)
            else :
              print('>>>找不到語音檔，請確定影音檔是否已準備好了\n')

          #將輸出的檔名列入結果清單中，方便後續壓縮、下載
          textFileList.append(getOutputTextFilename(outputFilename))
        #title = pList.title #供製作下載壓縮檔用的主檔名
        title = videoInfo['title']
      else :
        # Youtube single video
        #title = YouTube(urlOrFile).title #取得Youtube影片的標題字
        title = videoInfo['title']
        outputFilename = getFilenameFromTitle(title) #用影片的標題當主檔名，但需將不適合的字置換或去除
        #如果設定辨識過的重新辨識(overwrite_enable), 或是輸出字幕檔不存在就呼叫whisper
        if not isOutputTextFileExists(outputFilename) or overwrite_enable :
          # delete the old audio file
          removeAudioFile(filename)
          #getAudioFromYoutube(urlOrFile, tempFolder, filename)
          print('>> 下載影音檔案中...')
          try: getAudioFromYoutube_ydl(videoInfo['webpage_url'], tempFolder, filename)
          except:
            print('>>>無法由網址下載影音')
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
    else :
      #非Youtube的網址
      # delete the old audio file
      removeAudioFile(filename)
      isPlayList = False
      if re.search('drive\.google\.com', urlOrFile) :
        #Google drive files
        title = getYoutubeTitle_ydl(urlOrFile) #取得檔名
        outputFilename = getFilenameFromTitle(title) #主檔名，但需將不適合的字置換或去除
        if not isOutputTextFileExists(outputFilename) or overwrite_enable :
          getAudioFromYoutube_ydl(urlOrFile, tempFolder, filename)
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
      elif re.search('voca\.ro|vocaroo\.com', urlOrFile) :
        # Vocaroo or other web audio
        urlOrFile = getVocarooMP3URL(urlOrFile) #取得儲存 Vocaroo 的語音檔網址
        title = "vocaroo-"+os.path.basename(urlOrFile) #用 Vocaroo 的 id 當標題字
        outputFilename = title #用 vocaroo-xxxxxxxxxxx 當輸出的主檔名
        if not isOutputTextFileExists(outputFilename) or overwrite_enable :
          urllib.request.urlretrieve(urlOrFile, filename)
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
      else :
        # other website
        #os.remove(filename)        
        title = ''
        removeAudioFile(filename)
        print('\n>> 下載影音檔案中...\n')
        try : videoInfo = getYoutubePlaylistInfo_ydl(urlOrFile)
        except:
          print('\n>>> 無法下載該網址的影音資訊 ...\n')
        try: title = videoInfo['title'] #取得標題字
        except:
          title = 'unknow'
        outputFilename = getFilenameFromTitle(title) #用影片的標題當主檔名，但需將不適合的字置換或去除
        try: 
          getAudioFromYoutube_ydl(videoInfo['webpage_url'], tempFolder, filename)
        except:
          print('\n>>> 無法下載影音\n')
        #將輸出的檔名列入結果清單中，方便後續壓縮、下載
        textFileList.append(getOutputTextFilename(outputFilename))
        #! yt-dlp -q --force-overwrites -x --audio-format mp3 -o {audioFile} {url}
  else :
    #Colab儲存空間檔案處理(非網址的影音檔)
    isPlayList = False
    title = os.path.splitext(os.path.basename(urlOrFile))[0] #去掉資料夾名稱及附檔名，只取主檔名
    outputFilename = getFilenameFromTitle(title) #結果輸出檔的主檔名
    filename = urlOrFile #辨識指定的影音檔
    if os.path.exists(filename) :
      #將輸出的檔名列入結果清單中，方便後續壓縮、下載
      textFileList.append(getOutputTextFilename(outputFilename))
      #如果字幕檔已存在，而且不可覆蓋，就故意將語音檔設為不存在的檔名，這樣就不會重新辨識
      if (not overwrite_enable) and isOutputTextFileExists(outputFilename) :
        filename = 'fakeFilename.fake'
  #辨識非播放清單的單一語音檔
  if not isPlayList :
    print('\n>> '+title)
    #convert title to valid filename
    #outputFilename = slugify(title, allow_unicode=True, lowercase=False)
    #outputFilename = getFilenameFromTitle(title)
    # 語音檔存在的話，就進行辨識
    # load whisper model and get transcribe result
    print('\n>> 準備進行語音辨識...\n')
    if os.path.exists(filename) :
      print('>>>Whisper 辨識中...\n')
      transcribe(filename, output_path, outputFilename, outputFormat)
    else :
      print('>>>找不到語音檔，請確定影音檔是否已準備好了\n')
    #  start_downloading_immediately = False
  return title

#解析 .txt 檔案中的清單，一行行進行去抓音檔並進行辨識
def parseTxtFileAndTranscribe(txtFile) :
  if os.path.exists(txtFile) :
    file = open(txtFile, 'r')
    lines = file.readlines()
    file.close()
    #print(len(lines))
    for line in lines :
      line = re.sub('\r|\n', '', line, re.MULTILINE) #去掉所有換行字元
      getAudioAndTranscribe(line)
  else :
    print('找不到檔案: '+txtFile)

if (re.search('https\:\/\/', url) is None) and (not re.search('\.txt$', url, re.IGNORECASE) is None) :
  #解析 .txt 檔中的清單後，再進行辨識
  parseTxtFileAndTranscribe(url)
  title = os.path.splitext(os.path.basename(url))[0] #去掉資料夾名稱及附檔名，只取主檔名
else :
  #直接辨識 url 指定的網址或是檔案
  title = getAudioAndTranscribe(url)
  

#print(textFileList)
if len(textFileList)>0 and start_downloading_immediately :
  print('download...')
  #print(textFileList)
  filenames_list = ''
  for textFile in textFileList :
    #print(textFile)
    if os.path.exists(textFile) :
      convertNewLineFormat(textFile) # replace \n with \r\n (Windows format)
      if os.getcwd() != os.path.dirname(os.path.abspath(textFile)) :
        filename = os.path.basename(textFile)
        #copy textFile to current work directory
        ! cp {textFile} ./
      else :
        filename = textFile
      if filenames_list != '' :
        filenames_list = filenames_list+' '
      filenames_list = filenames_list+filename
  #print(filenames_list)

  if len(textFileList)>1 :
    print('\n壓縮並下載辨識結果')
    outputFilename = slugify(title, allow_unicode=True, lowercase=False)+'-'+outputFormat+'.zip'
    ! zip {outputFilename} -D {filenames_list}
  else :
    print('\n下載辨識結果')
    outputFilename = filenames_list

  google.colab.files.download(outputFilename)

  '''
  if isPlayList :
    print('\n壓縮並下載辨識結果')
    #title = pList.title
    outputFilename = slugify(title, allow_unicode=True, lowercase=False)+'-'+outputFormat+'.zip'
    #先將所有字幕檔的換行符號由 \n 換成 \r\n，全部壓縮
    ! zip -l {outputFilename} *.{outputFormat}
  else :
    print('\n下載辨識結果')
    outputFilename = outputFilename+'.'+outputFormat
  #下載辨識結果
  google.colab.files.download(outputFilename)
  '''

install yt_dlp ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 91.8 MB/s eta 0:00:00
>> 下載影音檔案中...

>> 世誠-看9Q和阿倪玩遊戲後爆氣關台

>> 準備進行語音辨識...

>>>Whisper 辨識中...



100%|████████████████████████████████████████| 461M/461M [00:03<00:00, 150MiB/s]
100%|██████████| 4177/4177 [00:08<00:00, 490.92frames/s]

download...

下載辨識結果


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 辨識結果比較

以同一個音檔
* https://voca.ro/15HVH0YvIaa6

用不同模型來進行辨識的結果
##Model small
<pre>
[00:00.000 --> 00:02.500] 大家午安
[00:02.500 --> 00:07.500] 科技的進步實在讓人很驚訝
[00:07.500 --> 00:13.000] 我們現在不怕利用廣的賣力
[00:13.000 --> 00:16.000] 我們說台語
[00:16.000 --> 00:19.000] 電腦跟我們說的台語就變異
[00:19.000 --> 00:23.000] 連鍵盤都不變硬
[00:23.000 --> 00:26.000] 現在我們來砌這個
[00:26.000 --> 00:28.500] OpenAI的Whisper
[00:28.500 --> 00:33.500] 也是跟我們說的微變異
[00:33.500 --> 00:36.500] 好,我們現在就來試試看
[00:36.500 --> 00:58.500] 看它到底有多厲害
</pre>

##Model medium
<pre>
[00:00.000 --> 00:03.000] 大家午安
[00:03.000 --> 00:08.000] 科技的進步實在讓人很驚險
[00:08.000 --> 00:11.000] 我們現在不用打字
[00:11.000 --> 00:14.000] 用講的也可以
[00:14.000 --> 00:16.000] 我們講台語
[00:16.000 --> 00:20.000] 電腦跟我們講的台語就變字
[00:20.000 --> 00:24.000] 連鍵盤都不用用
[00:24.000 --> 00:26.000] 現在我們來試這個
[00:26.000 --> 00:29.000] OpenAI的Whisper
[00:29.000 --> 00:33.000] 它可以跟我們講的話變字
[00:33.000 --> 00:37.000] 好,我們現在就來試看看
[00:37.000 --> 01:03.000] 看它到底多厲害
</pre>

基本上，精準度是用空間與時間換來的。